In [7]:
import csv
import re
import os
import numpy as np
import pandas as pd
import sys

In [16]:
sys.path.append('../..')

In [17]:
from utils import *

In [ ]:
def get_calibration_files(serial_nums,dirpath):
    """
    Function which gets all the calibration files associated with the
    instrument serial numbers.
    
    Args:
        serial_nums - serial numbers of the instruments
        dirpath - path to the directory containing the calibration files
    Returns:
        calibration_files - a dictionary of instrument uids with associated
            calibration files
    """
    calibration_files = {}
    for uid in serial_nums.keys():
        sn = serial_nums.get(uid)[0]
        sn = str(sn)
        files = []
        for file in os.listdir(dirpath):
            if sn in file:
                if 'Calibration_File' in file:
                    files.append(file)
                else:
                    pass
            else:
                pass
        
        calibration_files.update({uid:files})
        
    return calibration_files

In [ ]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/NUTNRB/'

In [5]:
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [25]:
excel_spreadsheet = '/home/andrew/Downloads/WHOI_Asset_Tracking.xlsx'

In [26]:
NUTNR = whoi_asset_tracking(spreadsheet=excel_spreadsheet,sheet_name=sheet_name,instrument_class='NUTNR',series='B')

In [27]:
NUTNR

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
876,NUTNR,B,239,115084,A00065,CGINS-NUTNRB-00239,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00004\n3305-00108-00048\n3305-00108...,3305-00308-00001,3305-00508-00040,3305-00900-00075\n3305-00900-00144\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,GI01SUMO-00001\nCP04OSSM-00006,NaN,NSIF,Reading High nitrate levels during calibration...
877,NUTNR,B,240,115085,A00066,CGINS-NUTNRB-00240,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00003\n3305-00108-00029\n3305-00108...,NaN,NaN,3305-00900-00008\n3305-00900-00231\n3305-00900...,WH-SC12-5-NUTNR-1001,2012-11-13 00:00:00,CP01CNSM-00001\nGS01SUMO-00002,NaN,NSIF,"09/2017: Clock issue - resets back to Jan 1, 2..."
878,NUTNR,B,260,115671,A00383,CGINS-NUTNRB-00260,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00010\n3305-00108-00056,NaN,3305-00508-00010,3305-00900-00109\n3305-00900-00317,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI Spare\nCP1 spare\nGS01SUMO-00001\nCP04OSSM-...,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
879,NUTNR,B,261,115672,A00384,CGINS-NUTNRB-00261,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00011\n3305-00108-00021\n3305-00108...,3305-00308-00007\n3305-00308-00031\n3305-00308...,3305-00508-00022\n3305-00508-00041,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,CP03ISSM-00002\nCP01CNSM-00005\nCP01CNSM-00007,NaN,NSIF,Returned to vendor 6/24/14 (RMA#2014-125)\nSen...
880,NUTNR,B,262,115673,A00385,CGINS-NUTNRB-00262,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00012\n3305-00108-00064,3305-00308-00002,NaN,3305-00900-00064\n3305-00900-00153\n3305-00900...,WH-SC12-05-NUTNR-1004,2013-08-12 00:00:00,GI01SUMO-00001,NaN,NaN,Sent to vedor as part of trade in for new (SUN...
881,NUTNR,B,266,116564,A00880,CGINS-NUTNRB-00266,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00015\n3305-00108-00040\n3305-00108...,3305-00308-00004\n3305-00308-00032,3305-00508-00024,3305-00900-00036\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP04OSSM-00001\nCP04OSSM-00004\nCP01CNSM-00008,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
882,NUTNR,B,267,116562,A00878,CGINS-NUTNRB-00267,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00018\n3305-00108-00053,NaN,3305-00508-00009,3305-00900-00109\n3305-00900-00363,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,GS01SUMO-00001\nCP01CNSM-00006,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
883,NUTNR,B,268,116563,A00879,CGINS-NUTNRB-00268,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00017\n3305-00108-00046\n3305-00108...,3305-00308-00003\n3305-00308-00034,3305-00508-00023,3305-00900-00071\n3305-00900-00173\n3305-00900...,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP03ISSM-00001\nCP03ISSM-00004\nCP03ISSM-00006,NaN,NSIF,4/2016: Scheduled to sample every half hour. S...
884,NUTNR,B,269,116565,A00881,CGINS-NUTNRB-00269,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00016\n3305-00108-00047\n3305-00114...,3305-00308-00005\n3305-00308-00009,3305-00508-00001,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-06-26 00:00:00,CP01CNSM-00002\nCP01CNSM-00003\nCP03ISSM-00005,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...
885,NUTNR,B,270,116899,A01142,CGINS-NUTNRB-00270,ISUS,1336-00014-00002,3.2.4,Satlantic,...,3305-00108-00020\n3305-00108-00050,3305-00308-00006\n3305-00308-00008,3305-00508-00038,3305-00900-00071\n3305-00900-00317,WH-SC12-05-NUTNR-1006,2014-09-24 00:00:00,CP3a Spare\nCP04OSSM-00002\nGI01SUMO-00003,NaN,NSIF,Sent to vedor as part of trade in for new (SUN...


In [ ]:
uids = list(set(NUTNR['UID']))
qct_dict = {}
for uid in uids:
    # Get the QCT Document numbers from the asset tracking sheet
    NUTNR['UID_match'] = NUTNR['UID'].apply(lambda x: True if uid in x else False)
    qct_series = NUTNR[NUTNR['UID_match'] == True]['QCT Testing']
    qct_series = list(qct_series.iloc[0].split('\n'))
    qct_dict.update({uid:qct_series})

In [ ]:
serial_nums = get_serial_nums(NUTNR, uids)
serial_nums

In [ ]:
cal_dict = get_calibration_files(serial_nums, cal_directory)
cal_dict

In [ ]:
filename = cal_dict['CGINS-NUTNRB-01065'][1]
filename

In [ ]:
cal_directory

In [ ]:
filepath = generate_file_path(cal_directory,filename.split('.')[0],ext=['.zip'])
filepath

In [ ]:
with ZipFile(filepath) as zfile:
    filename = [name for name in zfile.namelist() if name.endswith('.cal') and name.startswith('SNA')]
    if len(filename) == 1:
        data = zfile.read(filename[0]).decode('ASCII')
    elif len(filename) > 1:
        raise Exception(f'More than one calibration file found in {filepath}.')
    else:
        FileExistsError(f"No .cal file found in {filepath}")

In [ ]:
data

In [ ]:
for line in data.splitlines():
    print(line.split(','))

In [ ]:
serial = None
for line in data.splitlines():
    if line.startswith('H'):
        # Get the serial number
        if 'SUNA' in line and serial == None:
            _, info = line.split(',')
            serial = info.split()[1]
        # 

In [ ]:
serial

# Check the most recent NUTNR Cals

CGINS-NUTNRB-01089__20190327.csv
CGINS-NUTNRB-01090__20190401.csv
CGINS-NUTNRB-01091__20190327.csv

Project_Files > Records > Instrument_Records > NUTNR > NUTNR_Results

In [4]:
os.listdir('/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/')

['3305-00308-00023.xls',
 '3305-00108-00001-A.log',
 '3305-00108-00001.docx',
 '3305-00108-00002-A.log',
 '3305-00108-00002.docx',
 '3305-00108-00003-A.txt',
 '3305-00108-00003.docx',
 '3305-00108-00004-A.txt',
 '3305-00108-00004.docx',
 '3305-00108-00005-A.txt',
 '3305-00108-00005.doc',
 '3305-00108-00006-A.txt',
 '3305-00108-00006.doc',
 '3305-00108-00007-A.txt',
 '3305-00108-00007.doc',
 '3305-00108-00008-A.txt',
 '3305-00108-00009-A',
 '3305-00108-00009.doc',
 '3305-00108-00010-A.txt',
 '3305-00108-00010.docx',
 '3305-00108-00011-A.txt',
 '3305-00108-00011.docx',
 '3305-00108-00012-A.txt',
 '3305-00108-00012.docx',
 '3305-00108-00013-A.txt',
 '3305-00108-00013.doc',
 '3305-00108-00014-A.txt',
 '3305-00108-00014.doc',
 '3305-00108-00015-A.TXT',
 '3305-00108-00015.docx',
 '3305-00108-00016-A.TXT',
 '3305-00108-00017-A.TXT',
 '3305-00108-00017.docx',
 '3305-00108-00018-A.TXT',
 '3305-00108-00018.docx',
 '3305-00108-00019-A.log',
 '3305-00108-00019.docx',
 '3305-00108-00020-A.log',
 '3

In [28]:
NUTNR[NUTNR['Supplier\nSerial Number'] == 'NTR-1089']

,Instrument Class,Series,Supplier Serial Number,WHOI #,OOI #,UID,Model,CGSN PN,Firmware Version,Supplier,...,QCT Testing,PreDeployment,Post Deployment,Refurbishment/ Repair,DO Number,Date Received,Deployment History,Current Deployment,Instrument Location on Current Deployment,Notes
923,NUTNR,B,NTR-1089,118557,A02240,CGINS-NUTNRB-01089,SUNA,1336-00014-10002,2.5.1,Satlantic,...,3305-00127-00018,3305-00327-00055,NaN,NaN,NaN,2017-11-17 00:00:00,P11 spare,CP04OSSM-00010,NSIF,NaN


In [34]:
filepath = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/NUTNR/NUTNR_Results/3305-00327-00055'

In [35]:
from zipfile import ZipFile

In [39]:
for file in os.listdir(filepath):
    if '3305-00327-00055' in file and file.endswith('.zip'):
        print(file)
        zipfile = file

3305-00327-00055-A.zip


In [44]:
zfile = ZipFile(filepath+'/'+zipfile)

In [46]:
zfile.namelist()

['SNA1089-SelfTest-2019-03-27.txt',
 'SNA1089B.CAL',
 'SNA1089C.CAL',
 'SNA1089C.pdf',
 'SUNA-1089-Summary-20190327T192013.pdf']

In [52]:
data = zfile.read(zfile.namelist()[2]).decode('ASCII')

In [53]:
data

'H,SUNA 1089 Cal B  extinction coefficients and reference spectra\r\nH,File generated by SUNACom 1.2.3\r\nH,File format version 3\r\nH,File creation time 27-Mar-2019 19:03:55\r\nH,File generated by Internal Software Suite 1.9.21_243\r\nH,File format version 3\r\nH,File creation time 26-Oct-2017 11:54:12\r\nH,Operator JB\r\nH,PATH_LENGTH 10\r\nH,INT_PERIOD 350\r\nH,CONC_CAL_NO3 40.00\r\nH,CONC_CAL_SWA 34.92\r\nH,T_S_CORRECTABLE\r\nH,T_CAL 20.00\r\nH,T_CAL_SWA 20.00\r\nH,NitrateFile SUNA_1089_001.xml\r\nH,DIW Log file SUNA1089_Cal01_DIW_4.raw\r\nH,LNSW Log file SUNA1089_Cal01_LNSW_1.raw\r\nH,Nitrate in LNSW Log file SUNA1089_Cal01_NO3_1.raw\r\nH,Wavelength,nm\r\nH,NITRATE,uM\r\nH,AUX1,none\r\nH,AUX2,none\r\nH,Reference,counts\r\nH,Wavelength,NO3,SWA,TSWA,Reference\r\nE,189.89,0.00237930,0.00102376,0.00112028,15.62500000\r\nE,190.68,-0.00052801,0.00382779,0.00410855,20.66666667\r\nE,191.47,0.00014374,0.00301305,0.00317220,22.12500000\r\nE,192.26,0.00034292,0.00498207,0.00514493,18.7500000

In [54]:
print(data)

H,SUNA 1089 Cal B  extinction coefficients and reference spectra
H,File generated by SUNACom 1.2.3
H,File format version 3
H,File creation time 27-Mar-2019 19:03:55
H,File generated by Internal Software Suite 1.9.21_243
H,File format version 3
H,File creation time 26-Oct-2017 11:54:12
H,Operator JB
H,PATH_LENGTH 10
H,INT_PERIOD 350
H,CONC_CAL_NO3 40.00
H,CONC_CAL_SWA 34.92
H,T_S_CORRECTABLE
H,T_CAL 20.00
H,T_CAL_SWA 20.00
H,NitrateFile SUNA_1089_001.xml
H,DIW Log file SUNA1089_Cal01_DIW_4.raw
H,LNSW Log file SUNA1089_Cal01_LNSW_1.raw
H,Nitrate in LNSW Log file SUNA1089_Cal01_NO3_1.raw
H,Wavelength,nm
H,NITRATE,uM
H,AUX1,none
H,AUX2,none
H,Reference,counts
H,Wavelength,NO3,SWA,TSWA,Reference
E,189.89,0.00237930,0.00102376,0.00112028,15.62500000
E,190.68,-0.00052801,0.00382779,0.00410855,20.66666667
E,191.47,0.00014374,0.00301305,0.00317220,22.12500000
E,192.26,0.00034292,0.00498207,0.00514493,18.75000000
E,193.05,-0.00347507,0.00716433,0.00725705,24.54166667
E,193.84,-0.00002122,0.00344

In [61]:
coeffs = {'CC_di':[],
         'CC_eno3':[],
         'CC_eswa':[],
         'CC_wl':[],
         'CC_lower_wavelength_limit_for_spectra_fit':['217'],
         'CC_upper_wavelength_limit_for_spectra_fit':['240'],
         'CC_cal_temp':[]
         }

for line in data.splitlines():
    if 'T_CAL' in line:
        *ignore, cal_temp = line.split()
        coeffs['CC_cal_temp'].append(cal_temp)
    
    if line.startswith('E'):
        __, wl, eno3, eswa, __, di = line.split(',')
        coeffs['CC_di'].append(di)
        coeffs['CC_wl'].append(wl)
        coeffs['CC_eno3'].append(eno3)
        coeffs['CC_eswa'].append(eswa)
        
        
        

In [62]:
coeffs

{'CC_di': ['15.62500000',
  '20.66666667',
  '22.12500000',
  '18.75000000',
  '24.54166667',
  '25.29166667',
  '21.29166667',
  '23.66666667',
  '25.04166667',
  '23.62500000',
  '52.66666667',
  '181.95833333',
  '644.37500000',
  '1824.20833333',
  '3935.12500000',
  '6769.66666667',
  '9798.70833333',
  '12603.37500000',
  '14969.33333333',
  '16815.66666667',
  '18076.25000000',
  '18737.41666667',
  '18864.66666667',
  '18541.12500000',
  '17939.04166667',
  '17258.91666667',
  '16591.66666667',
  '16050.87500000',
  '15679.91666667',
  '15495.83333333',
  '15519.75000000',
  '15745.25000000',
  '16182.37500000',
  '16847.33333333',
  '17722.58333333',
  '18829.45833333',
  '20145.45833333',
  '21621.08333333',
  '23212.16666667',
  '24842.00000000',
  '26370.95833333',
  '27668.29166667',
  '28648.87500000',
  '29158.87500000',
  '29165.04166667',
  '28732.08333333',
  '27894.37500000',
  '26824.25000000',
  '25632.16666667',
  '24480.87500000',
  '23446.91666667',
  '22575.375

In [69]:
df = pd.DataFrame()
for i in coeffs.keys():
    df = df.append({
        'serial':'NTR-1089',
        'name':i,
        'value':coeffs.get(i),
        'notes':'stuff'
    }, ignore_index=True)

In [70]:
df

,name,notes,serial,value
0,CC_di,stuff,NTR-1089,"[15.62500000, 20.66666667, 22.12500000, 18.750..."
1,CC_eno3,stuff,NTR-1089,"[0.00237930, -0.00052801, 0.00014374, 0.000342..."
2,CC_eswa,stuff,NTR-1089,"[0.00102376, 0.00382779, 0.00301305, 0.0049820..."
3,CC_wl,stuff,NTR-1089,"[189.89, 190.68, 191.47, 192.26, 193.05, 193.8..."
4,CC_lower_wavelength_limit_for_spectra_fit,stuff,NTR-1089,[217]
5,CC_upper_wavelength_limit_for_spectra_fit,stuff,NTR-1089,[240]
6,CC_cal_temp,stuff,NTR-1089,"[20.00, 20.00]"
